In [2]:
import pandas as pd

time: 37.3 s


In [3]:
df = pd.read_csv('Capstone_oversampled.csv')

time: 7.19 s


In [4]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diag_1,diag_2,diag_3,max_glu_serum,...,rosiglitazone,insulin,change,diabetesMed,time_in_hospital,num_medications,number_diagnoses,num_ovrll_prcdrs,num_prvs_vists,readmitted
0,0.577032,0,0.198990,0.643167,0.566230,0.537816,0.616704,0.369356,0.365589,0.570449,...,0.571494,0.537379,0,1,-1.632460,-3.304858,-2.125874,-0.177270,-0.873587,0
1,0.577032,0,0.439961,0.588112,0.543250,0.610641,0.579307,0.662444,0.609798,0.570449,...,0.571494,0.645281,1,0,-0.271144,0.408817,0.865090,0.733542,-0.873587,1
2,0.569703,0,0.592636,0.588112,0.543250,0.610641,0.399032,0.510324,0.561094,0.570449,...,0.571494,0.537379,0,0,-0.825587,-0.237590,-0.909684,-1.431584,1.368394,0
3,0.577032,1,0.539074,0.588112,0.543250,0.610641,0.639189,0.662444,0.527934,0.570449,...,0.571494,0.645281,1,0,-0.825587,0.167145,-0.417314,0.024705,-0.873587,0
4,0.577032,1,0.550542,0.588112,0.543250,0.610641,0.455922,0.664514,0.536956,0.570449,...,0.571494,0.562352,1,0,-1.632460,-1.082723,-1.310201,0.328150,-0.873587,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164578,0.577032,0,0.548315,0.573521,0.562965,0.596606,0.666331,0.565511,0.510149,0.570449,...,0.571494,0.562352,1,0,0.766108,1.497467,0.865090,0.989198,1.133159,2
164579,0.577032,0,0.550542,0.572727,0.543250,0.543797,0.463905,0.621894,0.530579,0.569997,...,0.571494,0.562352,0,0,-0.825587,-0.333749,-1.310201,-0.899949,-0.873587,2
164580,0.577032,1,0.599916,0.588112,0.624512,0.610641,0.559822,0.591012,0.575650,0.570449,...,0.571494,0.564010,1,0,0.766108,-0.096210,0.865090,0.359390,-0.873587,2
164581,0.577032,1,0.570683,0.572727,0.157453,0.537816,0.594899,0.603862,0.529345,0.570449,...,0.571494,0.562352,0,0,2.086319,-0.383017,-0.926532,0.118653,1.497799,2


time: 848 ms


In [5]:
X = df.drop('readmitted',axis=1)
y = df['readmitted']

time: 91 ms


In [6]:
from pyforest import *

time: 7.81 s


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=96)

<IPython.core.display.Javascript object>

time: 13.9 s


In [8]:
from sklearn.pipeline import Pipeline

time: 132 ms


In [15]:
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from scipy.stats import randint
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.svm import SVC
import hyperopt as hp
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score



time: 37 ms


In [17]:
def auc_model(params):
    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
              'max_features': int(params['max_features']),
              'min_samples_leaf': int(params['min_samples_leaf'])}
    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()


params_space = {'n_estimators': hp.quniform('n_estimators', 0, 300, 50),
                'max_depth': hp.quniform('max_depth', 2, 30, 1),
                'max_features': hp.quniform('max_features', 1, 26, 1),
                'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 26, 1),
                'criterion': hp.choice('criterion', ['gini', 'entropy'])}
best = 0


def f(params):
    global best
    auc = auc_model(params)
    if auc > best:
        print('New Best', best, params)
    return {'loss': -auc, 'status': STATUS_OK}


trials = Trials()
best = fmin(f, params_space, algo=tpe.suggest, max_evals=30, trials=trials)
print('best:\n',best)

0%|          | 0/30 [00:15<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [20]:
from sklearn.model_selection import RandomizedSearchCV


time: 1.99 ms


In [21]:
rsearch = RandomizedSearchCV(pipe, r_param, cv=5, verbose=0,n_jobs=-1,random_state=96)

time: 2 ms


In [22]:
best_model_r = rsearch.fit(X_train,y_train)

time: 8min 26s


In [23]:
print(best_model_r.best_estimator_)
print("The mean accuracy of the model is through randomized search is :",best_model_r.score(X_test,y_test))

Pipeline(steps=[('classifier',
                 LGBMClassifier(max_depth=12, n_estimators=240,
                                num_leaves=22))])
The mean accuracy of the model is through randomized search is : 0.6383594936708861
time: 2.12 s


In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,classification_report


time: 2.01 ms


In [25]:
best_model_r.best_params_

{'classifier': LGBMClassifier(max_depth=12, n_estimators=240, num_leaves=22),
 'classifier__max_depth': 12,
 'classifier__n_estimators': 240,
 'classifier__num_leaves': 22}

time: 6 ms


In [26]:
rfc=RandomForestClassifier(**best_model_r.best_params_,random_state=96)
rfc.fit(X_train,y_train)
y_train_pred=rfc.predict(X_train)
y_train_prob=rfc.predict_proba(X_train)[:,1]

y_test_pred=rfc.predict(X_test)
y_test_prob=rfc.predict_proba(X_test)[:,1]

print('Confusion Matrix-Train\n',confusion_matrix(y_train,y_train_pred))
print('Accuracy Score-Train\n',accuracy_score(y_train,y_train_pred))
print('Classification Report-Train\n',classification_report(y_train,y_train_pred))

print('\n'*2)
print('Confusion Matrix-Test\n',confusion_matrix(y_test,y_test_pred))
print('Accuracy Score-Test\n',accuracy_score(y_test,y_test_pred))
print('Classification Report-Test\n',classification_report(y_test,y_test_pred))



TypeError: __init__() got an unexpected keyword argument 'classifier'